# Climate indices
We'll convert the monthly WorldClim data into monthly climate index maps, and use these to calculate further index values.

Of the WorldClim variables, temperature and humidity are most accurate (correlation > 0.99), followed by precipitation (0.86) and wind speed (0.76).

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import _functions as pmf

### 1. Find files

In [2]:
import glob

In [3]:
src_path = Path('/Users/wmk934/data/Global_geospatial/worldclim/')

In [4]:
# Units: https://worldclim.org/data/worldclim21.html
prec_files = sorted( glob.glob(str(src_path / 'wc2.1_30s_prec' / '*.tif')) ) # precipitation, mm
srad_files = sorted( glob.glob(str(src_path / 'wc2.1_30s_srad' / '*.tif')) ) # solar radiation, kJ m-2 day-1
tavg_files = sorted( glob.glob(str(src_path / 'wc2.1_30s_tavg' / '*.tif')) ) # temperature, degree C

### 2. Create the output directory

In [5]:
climate_path = Path(src_path) / 'derived' / 'climate_indices'
pet_path = Path(src_path) / 'derived' / 'pet'

In [6]:
climate_path.mkdir(exist_ok=True, parents=True)
pet_path.mkdir(exist_ok=True, parents=True)

### 3. Calculate PET and climate indices

#### 3.1 Functions

In [7]:
import numpy as np
import numpy.ma as ma
from osgeo import gdal, osr
import os

In [8]:
gdal.UseExceptions()

In [9]:
def calculate_pet_oudin(Re,Ta):

    # PE = Re / (lambda * rho) * (Ta + 5) / 100, if Ta+5 > 0
    # PE = 0, otherwise
    #
    # PE:     rate of potential evapotranspiration (mm day-1)
    # Re:     extraterrestrial radiation (MJ m-2 day-1)
    # lambda: latent heat flux (MJ kg-1)
    # rho:    density of water (kg m-3)
    # Ta:     mean daily air temperature (degree C)
    #
    # PE = Re / (lambda * rho) * (Ta + 5) / 100
    #    = (MJ m-2 day-1) / ((MJ kg-1) * (kg m-3)) * ((degree C) + (-)) / (-)
    #    = (MJ m-2 day-1) * 1 / (MJ m-3) * (degree C)
    #    = (MJ m-2 day-1) * (m3 MJ-1) # (degree C)
    #    = (m day-1 C-1)
    #
    # Oudin, L., Hervieu, F., Michel, C., Perrin, C., 
    # Andréassian, V., Anctil, F., & Loumagne, C. (2005). 
    # Which potential evapotranspiration input for a lumped 
    # rainfall–runoff model? Journal of Hydrology, 303(1-4), 
    # 290–306. doi:10.1016/j.jhydrol.2004.08.026

    # Assume WorldClim's 'srad' can be used in place of 'Re'
    rho = 1000 # (kg m-3)
    l = 2.45 # (MJ kg-1), lambda

    # Calculate PET
    pet = Re / (l*rho) * (Ta+5)/100
    pet[Ta+5 <= 0] = 0
    
    return pet

In [10]:
def get_geotif_data_as_array(file, band=1):
    ds = gdal.Open(file) # open the file
    band = ds.GetRasterBand(band) # get the data band
    data = band.ReadAsArray() # convert to numpy array for further manipulation   
    return data

In [11]:
def get_geotif_noData(src_file, band=1):
    src_ds = gdal.Open(src_file)
    src_band = src_ds.GetRasterBand(band)
    no_data_value = src_band.GetNoDataValue()
    src_ds = None
    return no_data_value

In [12]:
def write_geotif_sameDomain(src_file, des_file, des_data, no_data_value=None):
    
    # load the source file to get the appropriate attributes
    src_ds = gdal.Open(src_file)
    
    # get the geotransform
    des_transform = src_ds.GetGeoTransform()

    # Get the scale factor from the source metadata
    scale_factor = src_ds.GetRasterBand(1).GetScale()
    offset = src_ds.GetRasterBand(1).GetOffset()
    
    # get the data dimensions
    ncols = des_data.shape[1]
    nrows = des_data.shape[0]
    
    # make the file
    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(des_file,ncols,nrows,1,gdal.GDT_Float32, options = [ 'COMPRESS=DEFLATE' ])
    
    # Write the data
    #dst_ds.GetRasterBand(1).WriteArray( des_data )
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.WriteArray(des_data)
    if no_data_value:
        dst_band.SetNoDataValue(no_data_value)
    
    # Set the scale factor and offset in the destination band, if they were defined in the source
    if scale_factor: dst_ds.GetRasterBand(1).SetScale(scale_factor)
    if offset: dst_ds.GetRasterBand(1).SetOffset(offset)
    
    # Set the geotransform
    dst_ds.SetGeoTransform(des_transform)

    # Set the projection
    wkt = src_ds.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(wkt)
    dst_ds.SetProjection( srs.ExportToWkt() )
    
    # close files
    src_ds = None
    des_ds = None

    return

#### 3.2 Calculate PET

In [13]:
for month in range(1,13):

    # Easy access to files
    srad_file = srad_files[month-1] # [month-1] to account for 0-based indexing
    tavg_file = tavg_files[month-1]
    
    # Get the srad and tavg data
    srad = get_geotif_data_as_array(srad_file) 
    tavg = get_geotif_data_as_array(tavg_file)
    
    # Get the noData values
    srad_noData = get_geotif_noData(srad_file)
    tavg_noData = get_geotif_noData(tavg_file)
    
    # Mask the arrays to skip the noData cells
    srad_masked = ma.masked_equal(srad, srad_noData)
    tavg_masked = ma.masked_equal(tavg, tavg_noData)
    
    # Calculate PET
    pet = calculate_pet_oudin(srad_masked,tavg_masked)
    
    # Prep data for writing with known attributes
    pet_to_file = pet.filled()
    pet_noData = pet.fill_value.astype(pet.dtype) # noData value type must match array type for GDAL
    
    # Create output
    file_name = os.path.basename(srad_files[month-1]).replace('srad','pet_mm_per_day')
    pet_file = str(pet_path / file_name)
    write_geotif_sameDomain(tavg_files[month-1], pet_file, pet_to_file, no_data_value=pet_noData)

### 3.3 Calculate monthly snow and moisture index values

In [14]:
import calendar

In [15]:
# Get a list of the PET files
pet_files = sorted( glob.glob(str(pet_path / '*.tif')) ) # pet, mm day-1

In [17]:
# Prepare the output folders
snow_path = src_path / 'derived' / 'snow'
mois_path = src_path / 'derived' / 'moisture_index'

In [18]:
snow_path.mkdir(exist_ok=True, parents=True)
mois_path.mkdir(exist_ok=True, parents=True)

In [19]:
for month in range(1,13):
    print(f'Processing month {month}')

    # Easy access to files
    prec_file = prec_files[month-1] # [month-1] to account for 0-based indexing
    tavg_file = tavg_files[month-1]
    pet_file  = pet_files[month-1]
    
    # Get the datadata
    prec = get_geotif_data_as_array(prec_file) 
    tavg = get_geotif_data_as_array(tavg_file)
    pet  = get_geotif_data_as_array(pet_file)
    
    # Get the noData values
    prec_noData = get_geotif_noData(prec_file)
    tavg_noData = get_geotif_noData(tavg_file)
    pet_noData  = get_geotif_noData(pet_file)
    
    # Mask the arrays to skip the noData cells
    prec_masked = ma.masked_equal(prec, prec_noData)
    tavg_masked = ma.masked_equal(tavg, tavg_noData)
    pet_masked  = ma.masked_equal(pet,  pet_noData)
    
    # Convert pet [mm day-1] to [mm month-1] to match prec
    _,days_this_month = calendar.monthrange(2023,month) # year 2023 is as good a choice as any
    pet_masked = pet_masked * days_this_month
    
    # Compute the snow values
    snow = np.zeros_like(prec)
    snow = np.where(tavg<0, prec, 0)
    
    # Compute the aridity values
    mois = np.zeros_like(prec)
    mois = np.where(prec_masked > pet_masked, 1 - pet_masked/prec_masked, prec_masked/pet_masked-1)
    
    # Create masked arrays for saving
    snow_noData = float(-999)
    snow_to_file = ma.masked_array(snow, mask=snow<0, fill_value=snow_noData).filled()
    mois_noData = float(-999)
    mois_to_file = ma.masked_array(mois, mask=mois<-1, fill_value=mois_noData).filled()
    
    # Save to file
    snow_name = os.path.basename(prec_file).replace('prec','snow_mm_per_month')
    mois_name = os.path.basename(prec_file).replace('prec','moisture_index')
    snow_file = str(snow_path / snow_name)
    mois_file = str(mois_path / mois_name)
    
    write_geotif_sameDomain(prec_file, snow_file, snow_to_file, no_data_value=snow_noData)
    write_geotif_sameDomain(prec_file, mois_file, mois_to_file, no_data_value=mois_noData)

Processing month 1
Processing month 2
Processing month 3
Processing month 4
Processing month 5
Processing month 6
Processing month 7
Processing month 8
Processing month 9
Processing month 10
Processing month 11
Processing month 12


### 3.4 Calculate climate indices
See: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018WR022913

In [20]:
# Get a list of the snow and moisture index files
snow_files = sorted( glob.glob(str(snow_path / '*.tif')) ) # snow, mm month-1
mois_files = sorted( glob.glob(str(mois_path / '*.tif')) ) # moisture index, (-)

In [21]:
# Get the noData values
prec_noData = get_geotif_noData(prec_files[0])
snow_noData = get_geotif_noData(snow_files[0])
mois_noData = get_geotif_noData(mois_files[0])

In [22]:
# Get the data files we need as numpy stacks
prec_data = np.dstack( [get_geotif_data_as_array(file) for file in prec_files] )
snow_data = np.dstack( [get_geotif_data_as_array(file) for file in snow_files] )
mois_data = np.dstack( [get_geotif_data_as_array(file) for file in mois_files] )

In [23]:
# Convert into masked arrays
prec_data_masked = ma.masked_equal(prec_data, prec_noData)
snow_data_masked = ma.masked_equal(snow_data, snow_noData)
mois_data_masked = ma.masked_equal(mois_data, mois_noData)

In [24]:
# Compute the indices we want
fs = np.sum(snow_data_masked, axis=2) / np.sum(prec_data_masked, axis=2)

In [25]:
im = np.mean(mois_data_masked, axis=2)

In [26]:
imr = np.max(mois_data_masked, axis=2) - np.min(mois_data_masked, axis=2)

In [36]:
# Set the fill values to known values
fill_with = float(-999)
fs_to_file  = ma.masked_array(fs.filled(fill_value=fill_with),  mask=fs.mask,  fill_value=fill_with)
im_to_file  = ma.masked_array(im.filled(fill_value=fill_with),  mask=im.mask,  fill_value=fill_with)
imr_to_file = ma.masked_array(imr.filled(fill_value=fill_with), mask=imr.mask, fill_value=fill_with)

In [37]:
# Save to file
base_name = os.path.basename(prec_files[0])
fs_file = str(climate_path / base_name.replace('prec_01','climate_index_fs'))
im_file = str(climate_path / base_name.replace('prec_01','climate_index_im'))
imr_file= str(climate_path / base_name.replace('prec_01','climate_index_imr'))

In [38]:
write_geotif_sameDomain(prec_files[0], fs_file, fs_to_file, no_data_value=fill_with)
write_geotif_sameDomain(prec_files[0], im_file, im_to_file, no_data_value=fill_with)
write_geotif_sameDomain(prec_files[0], imr_file,imr_to_file,no_data_value=fill_with)